To use dash and Plotly, make sure to run these commands in a terminal to install the necessary packages:

pip install dash <br>
pip install Plotly <br>
pip install jupyter-dash <br>

Make sure to also install pandas if you need to with this command:

pip install pandas

In [2]:
!pip install jupyter-dash


In [3]:
import plotly
import dash
import jupyter_dash

# This cell should show you what version you have installed when executed
print("Plotly version:", plotly.__version__)
print("Dash version:", dash.__version__)
print("JupyterDash version:", jupyter_dash.__version__)


Plotly version: 5.22.0
Dash version: 2.18.2
JupyterDash version: 0.4.2


In [4]:
import numpy as np
import pandas as pd

print(np.__version__)
print(pd.__version__)


1.26.4
2.2.2


in terminal to run code below use pip install dash Plotly

In [6]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('super_covid19_data.csv')  # Place holder assuming super covid data

# Reshape the data: melt to have dates and cases in single columns
date_columns = [col for col in df.columns if '_cases' in col]
df_melted = df.melt(id_vars=['countyFIPS', 'County Name_x', 'State', 'StateFIPS_cases'],
                    value_vars=date_columns,
                    var_name='date',
                    value_name='cases')

# Convert date column to datetime and clean up date format
df_melted['date'] = pd.to_datetime(df_melted['date'].str.replace('_cases', ''))

# Filter to a specific state (e.g., NC) or set up the dashboard to allow state selection
# Initializing the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("COVID-19 Cases by County", style={'color': 'white'}),
    
    # Dropdown for selecting a state
    html.Label("Select State:", style={'color' : 'white'}),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df_melted['State'].unique()],
        value='NC'  # Default to North Carolina, can be any state in your data
    ),
    
    # Date range picker
    html.Label("Select Date Range:", style={'color' : 'white'}),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df_melted['date'].min(),
        max_date_allowed=df_melted['date'].max(),
        start_date=df_melted['date'].min(),
        end_date=df_melted['date'].max()
    ),
    
    
    # Radio items for scale type
    html.Label("Scale:", style={'colore' : 'white'}),
    dcc.RadioItems(
    id='scale-radio',
    options=[
        {'label': html.Span('Linear', style={'color': 'white'}), 'value': 'linear'},
        {'label': html.Span('Log', style={'color': 'white'}), 'value': 'log'}
    ],
    value='linear',
    labelStyle={'display': 'inline-block'}
),
    
    # Main graph
    dcc.Graph(id='covid-cases-graph')
])


# Callback to update the graph based on inputs
@app.callback(
    Output('covid-cases-graph', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('scale-radio', 'value')]
)
def update_graph(selected_state, start_date, end_date, scale_type):
    # Filter data based on state and date range
    filtered_df = df_melted[(df_melted['State'] == selected_state) &
                            (df_melted['date'] >= start_date) &
                            (df_melted['date'] <= end_date)]
    
    # Aggregate cases by date
    daily_cases = filtered_df.groupby('date')['cases'].sum().reset_index()

    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=daily_cases['date'], y=daily_cases['cases'],
        mode='lines', name='Daily Cases'
    ))
    
    # Apply scale (linear or log)
    fig.update_layout(
        yaxis_type=scale_type,
        title=f"COVID-19 Cases in {selected_state}",
        xaxis_title="Date",
        yaxis_title="Number of Cases"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


you might need to do pip install numpy<2    in your terminal

In [8]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression  # Example for linear model
from sklearn.preprocessing import PolynomialFeatures  # For non-linear model

# Load and prepare your data
df = pd.read_csv('super_covid19_data.csv')  

# Reshape the data to have dates and cases in single columns
date_columns = [col for col in df.columns if '_cases' in col]
df_melted = df.melt(id_vars=['countyFIPS', 'County Name_x', 'State', 'StateFIPS_cases'],
                    value_vars=date_columns,
                    var_name='date',
                    value_name='cases')
df_melted['date'] = pd.to_datetime(df_melted['date'].str.replace('_cases', ''))

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("COVID-19 Cases by County", style={'color': 'white'}),
    
    # Dropdown for selecting a state
    html.Label("Select State:", style={'color': 'white'}),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df_melted['State'].unique()],
        value='NC'  # Default to North Carolina, can be any state in your data
    ),
    
    # Date range picker
    html.Label("Select Date Range:", style={'color': 'white'}),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df_melted['date'].min(),
        max_date_allowed=df_melted['date'].max(),
        start_date=df_melted['date'].min(),
        end_date=df_melted['date'].max()
    ),
    
    # Radio items for scale type
    html.Label("Scale:", style={'color': 'white'}),
    dcc.RadioItems(
        id='scale-radio',
        options=[
            {'label': html.Span('Linear', style={'color': 'white'}), 'value': 'linear'},
            {'label': html.Span('Log', style={'color': 'white'}), 'value': 'log'}
        ],
        value='linear',
        labelStyle={'display': 'inline-block'}
    ),
    
    # Main graph
    dcc.Graph(id='covid-cases-graph')
])

# Function for generating linear predictions
def generate_linear_prediction(dates, cases):
    model = LinearRegression()
    dates_ordinal = np.array([date.toordinal() for date in dates]).reshape(-1, 1)
    model.fit(dates_ordinal, cases)
    predictions = model.predict(dates_ordinal)
    return predictions

# Function for generating non-linear predictions (polynomial, for example)
def generate_nonlinear_prediction(dates, cases, degree=3):
    poly = PolynomialFeatures(degree)
    dates_ordinal = np.array([date.toordinal() for date in dates]).reshape(-1, 1)
    dates_poly = poly.fit_transform(dates_ordinal)
    model = LinearRegression()
    model.fit(dates_poly, cases)
    predictions = model.predict(dates_poly)
    return predictions

# Callback to update the graph based on inputs
@app.callback(
    Output('covid-cases-graph', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('scale-radio', 'value')]
)
def update_graph(selected_state, start_date, end_date, scale_type):
    # Filter data based on state and date range
    filtered_df = df_melted[(df_melted['State'] == selected_state) &
                            (df_melted['date'] >= start_date) &
                            (df_melted['date'] <= end_date)]
    
    # Aggregate cases by date
    daily_cases = filtered_df.groupby('date')['cases'].sum().reset_index()

    # Generate trend line predictions
    if scale_type == 'linear':
        trend_line = generate_linear_prediction(daily_cases['date'], daily_cases['cases'])
    else:  # Using polynomial model for non-linear example
        trend_line = generate_nonlinear_prediction(daily_cases['date'], daily_cases['cases'])

    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=daily_cases['date'], y=daily_cases['cases'],
        mode='lines', name='Daily Cases'
    ))

    # Add the trend line to the plot
    fig.add_trace(go.Scatter(
        x=daily_cases['date'], y=trend_line,
        mode='lines', name='Trend Line (Prediction)',
        line=dict(dash='dash')
    ))
    
    # Apply scale (linear or log)
    fig.update_layout(
        yaxis_type=scale_type,
        title=f"COVID-19 Cases in {selected_state}",
        xaxis_title="Date",
        yaxis_title="Number of Cases"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [9]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('super_covid19_data.csv')  # Place holder assuming super covid data

# Reshape the data: melt to have dates and deaths in single columns
date_columns = [col for col in df.columns if '_deaths' in col]
df_melted = df.melt(id_vars=['countyFIPS', 'County Name_x', 'State', 'StateFIPS_deaths'],
                    value_vars=date_columns,
                    var_name='date',
                    value_name='deaths')

# Convert date column to datetime and clean up date format
df_melted['date'] = pd.to_datetime(df_melted['date'].str.replace('_deaths', ''))

# Filter to a specific state (e.g., NC) or set up the dashboard to allow state selection
# Initializing the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("COVID-19 Deaths by County", style={'color': 'white'}),
    
    # Dropdown for selecting a state
    html.Label("Select State:", style={'color' : 'white'}),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df_melted['State'].unique()],
        value='NC'  # Default to North Carolina, can be any state in your data
    ),
    
    # Date range picker
    html.Label("Select Date Range:", style={'color' : 'white'}),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df_melted['date'].min(),
        max_date_allowed=df_melted['date'].max(),
        start_date=df_melted['date'].min(),
        end_date=df_melted['date'].max()
    ),
    
    
    # Radio items for scale type
    html.Label("Scale:", style={'color' : 'white'}),
    dcc.RadioItems(
    id='scale-radio',
    options=[
        {'label': html.Span('Linear', style={'color': 'white'}), 'value': 'linear'},
        {'label': html.Span('Log', style={'color': 'white'}), 'value': 'log'}
    ],
    value='linear',
    labelStyle={'display': 'inline-block'}
),
    
    # Main graph
    dcc.Graph(id='covid-deaths-graph')
])


# Callback to update the graph based on inputs
@app.callback(
    Output('covid-deaths-graph', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('scale-radio', 'value')]
)
def update_graph(selected_state, start_date, end_date, scale_type):
    # Filter data based on state and date range
    filtered_df = df_melted[(df_melted['State'] == selected_state) &
                            (df_melted['date'] >= start_date) &
                            (df_melted['date'] <= end_date)]
    
    # Aggregate deaths by date
    daily_deaths = filtered_df.groupby('date')['deaths'].sum().reset_index()

    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=daily_deaths['date'], y=daily_deaths['deaths'],
        mode='lines', name='Daily Deaths'
    ))
    
    # Apply scale (linear or log)
    fig.update_layout(
        yaxis_type=scale_type,
        title=f"COVID-19 Deaths in {selected_state}",
        xaxis_title="Date",
        yaxis_title="Number of Deaths"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression  # Example for linear model
from sklearn.preprocessing import PolynomialFeatures  # For non-linear model

# Load and prepare your data
df = pd.read_csv('super_covid19_data.csv')  

# Reshape the data to have dates and deaths in single columns
date_columns = [col for col in df.columns if '_deaths' in col]
df_melted = df.melt(id_vars=['countyFIPS', 'County Name_x', 'State', 'StateFIPS_deaths'],
                    value_vars=date_columns,
                    var_name='date',
                    value_name='deaths')
df_melted['date'] = pd.to_datetime(df_melted['date'].str.replace('_deaths', ''))

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("COVID-19 Deaths by County", style={'color': 'white'}),
    
    # Dropdown for selecting a state
    html.Label("Select State:", style={'color': 'white'}),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df_melted['State'].unique()],
        value='NC'  # Default to North Carolina, can be any state in your data
    ),
    
    # Date range picker
    html.Label("Select Date Range:", style={'color': 'white'}),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df_melted['date'].min(),
        max_date_allowed=df_melted['date'].max(),
        start_date=df_melted['date'].min(),
        end_date=df_melted['date'].max()
    ),
    
    # Radio items for scale type
    html.Label("Scale:", style={'color': 'white'}),
    dcc.RadioItems(
        id='scale-radio',
        options=[
            {'label': html.Span('Linear', style={'color': 'white'}), 'value': 'linear'},
            {'label': html.Span('Log', style={'color': 'white'}), 'value': 'log'}
        ],
        value='linear',
        labelStyle={'display': 'inline-block'}
    ),
    
    # Main graph
    dcc.Graph(id='covid-deaths-graph')
])

# Function for generating linear predictions
def generate_linear_prediction(dates, deaths):
    model = LinearRegression()
    dates_ordinal = np.array([date.toordinal() for date in dates]).reshape(-1, 1)
    model.fit(dates_ordinal, deaths)
    predictions = model.predict(dates_ordinal)
    return predictions

# Function for generating non-linear predictions (polynomial, for example)
def generate_nonlinear_prediction(dates, deaths, degree=3):
    poly = PolynomialFeatures(degree)
    dates_ordinal = np.array([date.toordinal() for date in dates]).reshape(-1, 1)
    dates_poly = poly.fit_transform(dates_ordinal)
    model = LinearRegression()
    model.fit(dates_poly, deaths)
    predictions = model.predict(dates_poly)
    return predictions

# Callback to update the graph based on inputs
@app.callback(
    Output('covid-deaths-graph', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('scale-radio', 'value')]
)
def update_graph(selected_state, start_date, end_date, scale_type):
    # Filter data based on state and date range
    filtered_df = df_melted[(df_melted['State'] == selected_state) &
                            (df_melted['date'] >= start_date) &
                            (df_melted['date'] <= end_date)]
    
    # Aggregate deaths by date
    daily_deaths = filtered_df.groupby('date')['deaths'].sum().reset_index()

    # Generate trend line predictions
    if scale_type == 'linear':
        trend_line = generate_linear_prediction(daily_deaths['date'], daily_deaths['deaths'])
    else:  # Using polynomial model for non-linear example
        trend_line = generate_nonlinear_prediction(daily_deaths['date'], daily_deaths['deaths'])

    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=daily_deaths['date'], y=daily_deaths['deaths'],
        mode='lines', name='Daily Deaths'
    ))

    # Add the trend line to the plot
    fig.add_trace(go.Scatter(
        x=daily_deaths['date'], y=trend_line,
        mode='lines', name='Trend Line (Prediction)',
        line=dict(dash='dash')
    ))
    
    # Apply scale (linear or log)
    fig.update_layout(
        yaxis_type=scale_type,
        title=f"COVID-19 Deaths in {selected_state}",
        xaxis_title="Date",
        yaxis_title="Number of Deaths"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
